In [ ]:
import sys
import pymol
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

# load something into the PyMOL window
from pymol import cmd


Gtk-Message: 11:33:50.508: Failed to load module "pk-gtk-module"
Cannot open file '/home/vit/Projects/cryptic-nn/src/prediction-analysis/data/pymol/icons/icon2.svg', because: No such file or directory
Cannot open file '/home/vit/Projects/cryptic-nn/src/prediction-analysis/data/pymol/icons/icon2.svg', because: No such file or directory


Could not read PyMOL stylesheet.
DEBUG: PYMOL_DATA='./data'
 Detected OpenGL version 4.6. Shaders available.
 Geometry shaders not available
 Detected GLSL version 4.60.
PyMOL>set fetch_path, /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files
 Setting: fetch_path set to /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files.


In [ ]:
import os
import numpy as np

from biotite.structure import get_residues, get_residue_starts, spread_residue_wise
from biotite.structure.io.pdbx import get_structure
import biotite.structure.io.pdbx as pdbx
import biotite.database.rcsb as rcsb
import biotite
import csv

THRESHOLD = 0.8 # determined according to the F1 score (other thresholds were: [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.9])
DATA_PATH = '/home/vit/Projects/cryptic-nn/data/cryptobench'
# PREDICTIONS_PATH = f'{DATA_PATH}/smoothness-prediction'
PREDICTIONS_PATH = '/home/vit/Projects/cryptic-nn/data/predictions/ESM2-3B-extended-finetuning/smoothen-predictions'
CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'

with open('/home/vit/Projects/cryptic-nn/data/predictions/ESM2-3B-extended-finetuning/predictions.csv') as f:
    should_skip = True
    reader = csv.reader(f, delimiter=';')
    for row in reader:
        apo_structure = row[0]
        chain_id = row[1]
        protein_id = apo_structure + chain_id

        if protein_id != '5b0eB' and should_skip:
            continue
        should_skip = False
        
        binding_residues = [int(i[1:]) for i in row[3].split(' ')]

        predictions = np.load(f'{PREDICTIONS_PATH}/{protein_id}.npy', allow_pickle=True).item()
        predictions, indices = predictions['predictions'], predictions['indices']
        
        positive_indices = indices[predictions > THRESHOLD]
        negative_indices = indices[predictions <= THRESHOLD]

        cif_file_path = rcsb.fetch(apo_structure, "cif", target_path=CIF_FILES)
        cif_file = pdbx.CIFFile.read(cif_file_path)
        auth = get_structure(cif_file, model=1)

        auth = auth[
                (auth.chain_id == chain_id) &
                (biotite.structure.filter_amino_acids(auth))]
        auth_residues_only = auth[get_residue_starts(auth)]

        cmd.reinitialize()
        cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
        cmd.fetch(protein_id)
        cmd.zoom(protein_id)
        cmd.color('grey', protein_id)

        if len(positive_indices) > 0:
            cmd.color('green', f'{protein_id} and resid {"+".join([str(i) for i in auth_residues_only.res_id[positive_indices]])}')
        cmd.color('red', f'{protein_id} and resid {"+".join([str(i) for i in auth_residues_only.res_id[negative_indices]])}')

        intersection = np.intersect1d(binding_residues, positive_indices)
        if len(intersection) > 0:
            cmd.color('lightblue', f'{protein_id} and resid {"+".join([str(i) for i in auth_residues_only.res_id[intersection]])}')
        cmd.color('blue', f'{protein_id} and resid {"+".join([str(i) for i in auth_residues_only.res_id[binding_residues]])}')
        cmd.show('surface')
        decision = input(">Press Enter for the next protein...\n")
        if decision == 'q':
            break

 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
